In [1]:
import os
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage
from IPython.display import display, Markdown, Latex
from datetime import datetime

In [2]:
API_KEY = os.environ['MISTRAL_API_KEY']

MODEL_ID = 'open-mixtral-8x22b'
MODEL_SEED = 2
MODEL_TEMPERATURE = 0.7

file_system_prompt_1 = open("../../system_prompt_1.txt", "r")
file_system_prompt_2 = open("../../system_prompt_2.txt", "r")
SYSTEM_PROMPT_1 = file_system_prompt_1.read()
SYSTEM_PROMPT_2 = file_system_prompt_2.read()
file_system_prompt_1.close()
file_system_prompt_2.close()

FILE_PATH_PROBLEM = '../../../../Datasets/IP_4_Stemez040177/'
assert(FILE_PATH_PROBLEM != '../../../../Datasets/')
file_problem_description = open(FILE_PATH_PROBLEM + 'ProblemDescription.txt', 'r')
PROBLEM_DESCRIPTION = file_problem_description.read()
file_problem_description.close()


client = MistralClient(api_key=API_KEY)

print(f'Time of execution: {datetime.now()}')

Time of execution: 2024-05-22 11:30:48.440648


## Step 1 - Generate Mathematical Formulation 

In [3]:
messages_1 = [
    ChatMessage(role="system", content=SYSTEM_PROMPT_1),
    ChatMessage(role="user", content=PROBLEM_DESCRIPTION)
]

In [4]:
response_1 = client.chat(
    model=MODEL_ID,
    messages=messages_1,
    random_seed=MODEL_SEED,
    temperature=MODEL_TEMPERATURE 
)

response_1_text = response_1.choices[0].message.content

In [5]:
Markdown(response_1_text)

To formulate a mathematical optimization model for this problem, we will define the parameters, decision variables, objective function, and constraints as follows:

Parameters:
- P1: Priority factor for sales goal deviation
- P2: Priority factor for overtime goal deviation (full-time employees)
- P3: Priority factor for employee hours goal deviation (full-time employees)
- P4: Priority factor for employee hours goal deviation (part-time employees) and overtime goal deviation (part-time employees)
- W1: Relative weight for full-time employee hours (2 times)
- W2: Relative weight for part-time employee overtime hours (3 times)
- S_target: Sales target (5500 units)
- F\_hrs_target: Full-time employee hours target (800 hours)
- P\_hrs_target: Part-time employee hours target (320 hours)
- F\_overtime_target: Full-time employee overtime target (100 hours)
- S_rate_f: Sales rate for full-time employees (5 records per hour)
- S_rate_p: Sales rate for part-time employees (2 records per hour)

Decision Variables:
- F_hrs: Full-time employee hours
- P_hrs: Part-time employee hours
- F_overtime: Full-time employee overtime hours
- P_overtime: Part-time employee overtime hours
- S: Sales (number of units)

Objective Function:
Minimize the weighted sum of deviations:
Z = P1 * |S - S_target| + P2 * |F_overtime - F_overtime_target| + P3 * W1 * |F_hrs - F\_hrs_target| + P4 * |P_hrs - P\_hrs_target| + P4 * W2 * P_overtime

Constraints:
1. Sales constraint: S = F_hrs * S_rate_f + P_hrs * S_rate_p
2. Full-time employee hours constraint: F_hrs >= 0
3. Part-time employee hours constraint: P_hrs >= 0
4. Full-time employee overtime constraint: F_overtime >= 0
5. Part-time employee overtime constraint: P_overtime >= 0
6. Full-time employee overtime target constraint: F_overtime <= F_overtime_target
7. Non-negativity constraints: S, F_hrs, P_hrs, F_overtime, P_overtime >= 0

This optimization model will help you find the best balance between sales targets, employee hours, and overtime, considering the priorities and weights of each deviation. You can then use the decision variables (F\_hrs, P\_hrs, F\_overtime, P\_overtime, and S) to make informed decisions about managing your employees' working hours and sales targets.

In [6]:
print(response_1_text)

To formulate a mathematical optimization model for this problem, we will define the parameters, decision variables, objective function, and constraints as follows:

Parameters:
- P1: Priority factor for sales goal deviation
- P2: Priority factor for overtime goal deviation (full-time employees)
- P3: Priority factor for employee hours goal deviation (full-time employees)
- P4: Priority factor for employee hours goal deviation (part-time employees) and overtime goal deviation (part-time employees)
- W1: Relative weight for full-time employee hours (2 times)
- W2: Relative weight for part-time employee overtime hours (3 times)
- S_target: Sales target (5500 units)
- F\_hrs_target: Full-time employee hours target (800 hours)
- P\_hrs_target: Part-time employee hours target (320 hours)
- F\_overtime_target: Full-time employee overtime target (100 hours)
- S_rate_f: Sales rate for full-time employees (5 records per hour)
- S_rate_p: Sales rate for part-time employees (2 records per hour)

D

## Step 2 - Generate the Pyomo Code

In [7]:
messages_2 = [
    ChatMessage(role="system", content=SYSTEM_PROMPT_2),
    ChatMessage(role="user", content=response_1_text)
]

In [8]:
response_2 = client.chat(
    model=MODEL_ID,
    messages=messages_2,
    random_seed=MODEL_SEED,
    temperature=MODEL_TEMPERATURE
)

response_2_text = response_2.choices[0].message.content

In [9]:
Markdown(response_2_text)

Here's the Python Pyomo code for your optimization problem. I've used sample data where indicated in the code comments.

```python
from pyomo.environ import *

# Define the model
model = ConcreteModel()

# Define the parameters (sample data)
model.P1 = 1
model.P2 = 1
model.P3 = 1
model.P4 = 1
model.W1 = 2
model.W2 = 3
model.S_target = 5500
model.F_hrs_target = 800
model.P_hrs_target = 320
model.F_overtime_target = 100
model.S_rate_f = 5
model.S_rate_p = 2

# Define the decision variables
model.F_hrs = Var(domain=NonNegativeReals)
model.P_hrs = Var(domain=NonNegativeReals)
model.F_overtime = Var(domain=NonNegativeReals)
model.P_overtime = Var(domain=NonNegativeReals)
model.S = Var(domain=NonNegativeReals)

# Define the objective function
def obj_rule(model):
    return (model.P1 * abs(model.S - model.S_target) +
            model.P2 * abs(model.F_overtime - model.F_overtime_target) +
            model.P3 * model.W1 * abs(model.F_hrs - model.F_hrs_target) +
            model.P4 * abs(model.P_hrs - model.P_hrs_target) +
            model.P4 * model.W2 * model.P_overtime)

model.obj = Objective(rule=obj_rule, sense=minimize)

# Define the constraints
def sales_constraint_rule(model):
    return model.S == model.F_hrs * model.S_rate_f + model.P_hrs * model.S_rate_p

model.sales_constraint = Constraint(rule=sales_constraint_rule)

def full_time_employee_hours_constraint_rule(model):
    return model.F_hrs >= 0

model.full_time_employee_hours_constraint = Constraint(rule=full_time_employee_hours_constraint_rule)

def part_time_employee_hours_constraint_rule(model):
    return model.P_hrs >= 0

model.part_time_employee_hours_constraint = Constraint(rule=part_time_employee_hours_constraint_rule)

def full_time_employee_overtime_constraint_rule(model):
    return model.F_overtime >= 0

model.full_time_employee_overtime_constraint = Constraint(rule=full_time_employee_overtime_constraint_rule)

def part_time_employee_overtime_constraint_rule(model):
    return model.P_overtime >= 0

model.part_time_employee_overtime_constraint = Constraint(rule=part_time_employee_overtime_constraint_rule)

def full_time_employee_overtime_target_constraint_rule(model):
    return model.F_overtime <= model.F_overtime_target

model.full_time_employee_overtime_target_constraint = Constraint(rule=full_time_employee_overtime_target_constraint_rule)

# Solve the model
solver = SolverFactory('glpk')
solver.solve(model)

# Print the results
print("F_hrs: ", model.F_hrs.value)
print("P_hrs: ", model.P_hrs.value)
print("F_overtime: ", model.F_overtime.value)
print("P_overtime: ", model.P_overtime.value)
print("S: ", model.S.value)
```

This code sets up the optimization model using Pyomo and solves it using the GLPK solver. You can change the parameters and decision variables as needed.

In [10]:
print(response_2_text)

Here's the Python Pyomo code for your optimization problem. I've used sample data where indicated in the code comments.

```python
from pyomo.environ import *

# Define the model
model = ConcreteModel()

# Define the parameters (sample data)
model.P1 = 1
model.P2 = 1
model.P3 = 1
model.P4 = 1
model.W1 = 2
model.W2 = 3
model.S_target = 5500
model.F_hrs_target = 800
model.P_hrs_target = 320
model.F_overtime_target = 100
model.S_rate_f = 5
model.S_rate_p = 2

# Define the decision variables
model.F_hrs = Var(domain=NonNegativeReals)
model.P_hrs = Var(domain=NonNegativeReals)
model.F_overtime = Var(domain=NonNegativeReals)
model.P_overtime = Var(domain=NonNegativeReals)
model.S = Var(domain=NonNegativeReals)

# Define the objective function
def obj_rule(model):
    return (model.P1 * abs(model.S - model.S_target) +
            model.P2 * abs(model.F_overtime - model.F_overtime_target) +
            model.P3 * model.W1 * abs(model.F_hrs - model.F_hrs_target) +
            model.P4 * abs(mod

### Code Executability

In [11]:
from pyomo.environ import *

# Define the model
model = ConcreteModel()

# Define the parameters (sample data)
model.P1 = 1
model.P2 = 1
model.P3 = 1
model.P4 = 1
model.W1 = 2
model.W2 = 3
model.S_target = 5500
model.F_hrs_target = 800
model.P_hrs_target = 320
model.F_overtime_target = 100
model.S_rate_f = 5
model.S_rate_p = 2

# Define the decision variables
model.F_hrs = Var(domain=NonNegativeReals)
model.P_hrs = Var(domain=NonNegativeReals)
model.F_overtime = Var(domain=NonNegativeReals)
model.P_overtime = Var(domain=NonNegativeReals)
model.S = Var(domain=NonNegativeReals)

# Define the objective function
def obj_rule(model):
    return (model.P1 * abs(model.S - model.S_target) +
            model.P2 * abs(model.F_overtime - model.F_overtime_target) +
            model.P3 * model.W1 * abs(model.F_hrs - model.F_hrs_target) +
            model.P4 * abs(model.P_hrs - model.P_hrs_target) +
            model.P4 * model.W2 * model.P_overtime)

model.obj = Objective(rule=obj_rule, sense=minimize)

# Define the constraints
def sales_constraint_rule(model):
    return model.S == model.F_hrs * model.S_rate_f + model.P_hrs * model.S_rate_p

model.sales_constraint = Constraint(rule=sales_constraint_rule)

def full_time_employee_hours_constraint_rule(model):
    return model.F_hrs >= 0

model.full_time_employee_hours_constraint = Constraint(rule=full_time_employee_hours_constraint_rule)

def part_time_employee_hours_constraint_rule(model):
    return model.P_hrs >= 0

model.part_time_employee_hours_constraint = Constraint(rule=part_time_employee_hours_constraint_rule)

def full_time_employee_overtime_constraint_rule(model):
    return model.F_overtime >= 0

model.full_time_employee_overtime_constraint = Constraint(rule=full_time_employee_overtime_constraint_rule)

def part_time_employee_overtime_constraint_rule(model):
    return model.P_overtime >= 0

model.part_time_employee_overtime_constraint = Constraint(rule=part_time_employee_overtime_constraint_rule)

def full_time_employee_overtime_target_constraint_rule(model):
    return model.F_overtime <= model.F_overtime_target

model.full_time_employee_overtime_target_constraint = Constraint(rule=full_time_employee_overtime_target_constraint_rule)

# Solve the model
solver = SolverFactory('glpk')
solver.solve(model)

# Print the results
print("F_hrs: ", model.F_hrs.value)
print("P_hrs: ", model.P_hrs.value)
print("F_overtime: ", model.F_overtime.value)
print("P_overtime: ", model.P_overtime.value)
print("S: ", model.S.value)

RuntimeError: Cannot write legal LP file.  Objective 'obj' has nonlinear terms that are not quadratic.

### Solution Correctness